## ループで福岡全域のデータを取得

In [5]:
import requests
import json
import pandas as pd
import time
import numpy as np


#地点名から緯度と経度を取得する関数
def Get_GioCode(place, api_key):
    # url = 'https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}'.format(place, api_key)
    # response = requests.get(url)
    # data = json.loads(response.text)
    # latitude = data['results'][0]['geometry']['location']['lat']
    # longitude = data['results'][0]['geometry']['location']['lng']
    # location = str(latitude) + ',' + str(longitude)
    # return location
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': place, 'key': api_key}
    res = requests.get(url, params=params)
    data = res.json()
    # 緯度と経度を取得
    lat = data['results'][0]['geometry']['location']['lat']
    lng = data['results'][0]['geometry']['location']['lng']
    location = str(lat) + ',' +str(lng)
    return location

#条件から周辺施設を検索する関数
def Get_GioData(location, keyword, api_key, radius=50000):
    result = []
    next_page_token = None
    while True:
        if next_page_token:
            time.sleep(3)
            url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&keyword={keyword}&language=ja&key={api_key}&pagetoken={next_page_token}'
        else:
            time.sleep(3)
            url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&keyword={keyword}&language=ja&key={api_key}'
        response = requests.get(url)
        data = response.json()
        result += data['results']
        next_page_token = data.get('next_page_token')
        if not next_page_token:
            break
    return result

## コーディング

In [7]:
# Google PlaceのAPIキーを入力
api_key = ''

#k検索する地点名の候補をリストに渡す
# location_name = ['博多駅', '西鉄天神駅','西鉄大橋駅','福岡自動車運転免許試験場','南福岡特別支援学校','福工大前駅','福岡市 貝塚駅',
#                  '博多南駅','マリンワールド' ,'唐人町駅','ビッグモーター西福岡店', '福岡市立早良中学校','福岡市 青葉公園','福岡市 西部運動公園',
#                  '姪浜駅', '九大学研都市駅', '福大前駅', '福岡海釣り公園', '博多の森','南福岡駅' ]

lat_range = [33.5, 33.6]
lng_range = [130.25, 130.5]

lat_grid, lng_grid = np.meshgrid(np.linspace(lat_range[0], lat_range[1], num=6), np.linspace(lng_range[0], lng_range[1], num=6))

#検索地点からの検索範囲を設定(m)
radius = 3000 

#今回の検索したい施設のキーワードを入力
keyword = 'ジム'

#検索した後に整形した必要なデータの格納先
result_dic_data = []

# 博多駅の座標を取得
for i in range(lat_grid.shape[0]):
    for j in range(lng_grid.shape[1]):
        location = str(lat_grid[i,j]) + ',' + str(lng_grid[i,j])
        results = Get_GioData(location, keyword, api_key, radius)

    #抽出データから必要なカラムを辞書形式で取得   
        for result in results:
            place = {}
            place["name"] = result["name"]
            place["address"] = result["vicinity"]
            place["lat"] = result["geometry"]["location"]["lat"]
            place["lng"] = result["geometry"]["location"]["lng"]
            place['rating'] = result['rating']
            place['user_ratings_total'] = result['user_ratings_total']
            result_dic_data.append(place)

#データフレームに変換
df = pd.DataFrame(result_dic_data)

#住所情報から市と区のカラムを取得
df["city"] = df["address"].apply(lambda x: x.split("市")[0] if "市" in x else None)
df["district"] = df["address"].apply(lambda x: x.split("区")[0].split("市")[-1] if "区" in x else None)
df['city-district'] = '福岡市' + df['district'] + '区'
df['search_word'] = keyword

#検索で出た重複を排除
df = df.drop_duplicates('name').reset_index(drop=True)


#csvファイルに出力
df.to_csv('福岡市周辺の' + keyword + 'に関する位置情報.csv')

In [317]:
extrct_example = {'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 33.5928065, 'lng': 130.3855521},
  'viewport': {'northeast': {'lat': 33.59412092989271,
    'lng': 130.3871242798927},
   'southwest': {'lat': 33.59142127010727, 'lng': 130.3844246201072}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
 'icon_background_color': '#7B9EB0',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
 'name': '福岡市中央区24時間営業パーソナルジム【VISION24 福岡天神店】',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 3888,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102376585050495911523">A Google User</a>'],
   'photo_reference': 'AfLeUgNpwUPchx0Q45qhb0chkiJ9Cbe0IDNd0KvuEFGj807adaxSHcAwjkf4xl0aqxGOi4fFp4DY4YiYRdBaYihL3EwuhUHkhXMNW5LhZ1tEjTddF0ExMV94-K2Zqeu6KbeaR-kHaiNrZ1ZN1sD903ymlWrwaryhnKbWBiPO20htG9e7sDvF',
   'width': 5184}],
 'place_id': 'ChIJ_WZOWBGTQTUROlTqEPeYdjQ',
 'plus_code': {'compound_code': 'H9VP+46 福岡市, 日本、福岡県',
  'global_code': '8Q5GH9VP+46'},
 'rating': 4.8,
 'reference': 'ChIJ_WZOWBGTQTUROlTqEPeYdjQ',
 'scope': 'GOOGLE',
 'types': ['gym', 'health', 'store', 'point_of_interest', 'establishment'],
 'user_ratings_total': 269,
 'vicinity': '福岡市中央区港１丁目１０−５ キテラタウン長浜 ３F'}

## Appendix

In [323]:
import requests
import pandas as pd


#地点検索の緯度と経度のデータフレーム
latitude = []
longitude = []

for place in location_name:
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={place}&key={api_key}"
    response = requests.get(url).json()
    latitude.append(response['results'][0]['geometry']['location']['lat'])
    longitude.append(response['results'][0]['geometry']['location']['lng'])

df_loc = pd.DataFrame({'latitude': latitude, 'longitude': longitude, 'place_name':location_name})

df_loc.to_csv('検索地点の情報.csv')